# Práctica 6

Modelo Neuronal de Bengio


In [1]:
# General
import os
import time
import shutil
import random
from typing import Tuple
from argparse import Namespace  # Guardar variables y parámetros
import matplotlib.pyplot as plt

# Preprocesamiento
import nltk
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import numpy as np

# PyTorch
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.nn.functional as F

# scikit-learn
from sklearn.metrics import accuracy_score

# tqmd
from tqdm.notebook import tqdm

Variables generales para reproducibilidad


In [2]:
seed = 1111
random.seed(seed)  # python seed
np.random.seed(seed)  # numpy seed
torch.manual_seed(seed)  # torch seed
torch.backends.cudnn.benchmark = False

In [3]:
# Función : get_texts_from_file()
def get_texts_from_file(path_corpus, path_truth):
  """
  Función para leer los archivos de tuits. Cada línea (tuit) será un elemento de la lista.
  """
  tr_txt = []
  tr_y = []
  with open(path_corpus) as f_corpus, open(path_truth) as f_truth:
    for tweet in f_corpus:
      tr_txt.append(tweet)
    for label in f_truth:
      tr_y.append(label)
  return tr_txt, tr_y


PATH = "../../corpus/"

X_train, y_train = get_texts_from_file(
  PATH + "/mex20_train.txt",
  PATH + "/mex20_train_labels.txt")

X_val, y_val = get_texts_from_file(
  PATH + "/mex20_val.txt",
  PATH + "/mex20_val_labels.txt")

y_train = list(map(int, y_train))
y_val = list(map(int, y_val))

In [4]:
args = Namespace()
args.N = 4

¡! En la parte en la que se usa la matriz de embeddings (dentro de `NgramData.fit()`) se le asocia un atributo `.vector_size` que hast ahora no se ha definido. Checa eso.


In [5]:
class NgramData():
  """
  Esta clase toma un corpus y a través de los métodos fit y transform, se crea una lista de 
  n-gramas pensada para el entrenamiento de la red neuronal de Bengio pensando en una CBOW.
  """

  def __init__(self,
               N: int,
               vocab_max: int = 5000,
               tokenizer: callable = None,
               embeddings: np.ndarray = None):
    """
    Constructor de la clase.

    Args:
        N (int): Tamaño de los n-gramas.
        vocab_max (int, optional): Tamaño máximo del vocabulario a considerar. Defaults to 5000.
        tokenizier (callable, optional): Tokenizador. Defaults to None.
        embeddings (np.ndarray, optional): Matriz de embeddings pre-entrenada. Debe entrar en el orden en el que entran las palabras. Defaults to None.
    """
    self.N = N
    self.vocab_max = vocab_max
    self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
    self.embeddings = embeddings

    # Tokens que no queremos en nuestro corpus.
    self.punct = ['.', ',', ';', ':', '-', '^', '"'
                  '"', '!', '¡', '¿', '?', '<url>', '#', '@usuario']

    # Tokens especiales
    self.UNK = "<unk>"
    self.SOS = "<s>"
    self.EOS = "</s>"

  def get_vocab_size(self) -> int:
    """
    Devuelve el tamaño del vocabulario.

    Returns:
        int: Tamaño del vocabulario.
    """
    return len(self.vocab)

  def default_tokenizer(self, doc: str) -> list:
    """
    Tokenizador por defecto. Simplemente separa cada oración por espacios.

    Args:
        doc (str): Documento a tokenizar.

    Returns:
        list: Lista de tokens.
    """
    return doc.split(" ")

  def remove_word(self, word: str) -> bool:
    """
    Verifica si la palabra en cuestión debe eliminarse según los siguientes criterios:
    - Es un signo de puntuación
    - Es un dígito

    Args:
        word (str): Palabra a evaluar.

    Returns:
        bool: True si se elimina.
    """
    word = word.lower()
    is_punct = True if word in self.punct else False
    is_digit = word.isnumeric()
    return is_punct or is_digit

  def sortFreqDist(self, freq_dist: nltk.FreqDist) -> list:
    """
    Devuelve una lista con el top de palabras por frecuencia. El tamaño de la lista es self.vocab_max.

    Args:
        freq_dist (nltk.FreqDist): Objeto de frecuencias (nltk) del corpus considerado.

    Returns:
        list: Lista de tamaño self.vocab_max.
    """
    freq_dist = dict(freq_dist)
    # Aquí key es una función que se aplica a cada parámetro
    # antes de compararlo. En este caso se pasa
    # freq_dist.get para asegurarse de que el ordenamiento
    # se haga por las frecuencias y no por orden alfabético.
    return sorted(freq_dist,
                  key=freq_dist.get,
                  reverse=True)

  def get_vocab(self, corpus: list[str]) -> set:
    """
    Devuelve el vocabulario a partir de un corpus dado.

    Args:
        corpus (list[str]): Corpus del cual se quiere obtener el vocabulario. Lista de documentos.

    Returns:
        set: Vocabulario.
    """
    freq_dist = FreqDist(
      [w.lower()
       for sentence in corpus
       for w in self.tokenizer(sentence)
       if not self.remove_word(w)]
    )
    sorted_words = self.sortFreqDist(freq_dist)[:self.vocab_max-3]
    return set(sorted_words)

  def fit(self, corpus: list[str]) -> None:
    """
    Carga el vocabulario y crea diccionarios de índices <-> palabras. Además, si se aporta una matriz de embeddings pre-entrenados, también construye la submatriz con los elementos del vocabulario.

    Args:
        corpus (list[str]): Lista de documentos.
    """
    # Cargamos el vocabulario
    self.vocab = self.get_vocab(corpus)
    self.vocab.add(self.UNK)
    self.vocab.add(self.SOS)
    self.vocab.add(self.EOS)

    # Diccionarios palabras <-> ids
    self.w2id = dict()
    self.id2w = dict()

    if self.embeddings:
      self.embeddings_matrix = np.empty([self.vocab_max,
                                         self.embeddings.vector_size])

    id = 0
    for doc in corpus:
      for word in self.tokenizer(doc):
        word_ = word.lower()
        if (word_ in self.vocab) and (not word_ in self.w2id):
          self.w2id[word_] = id
          self.id2w[id] = word_

          # Si se aporta una matriz de embeddings,
          # aquí se crea la submatriz.
          if self.embeddings:
            if word in self.embeddings:
              self.embeddings_matrix[id] = self.embeddings[word_]
            else:
              self.embeddings_matrix[id] = np.random.rand(
                self.embeddings.vector_size)

          id += 1

    # Añadirmos los tokens especiales a los diccionarios.
    self.w2id.update(
      {self.UNK: id,
       self.SOS: id + 1,
       self.EOS: id + 2}
    )
    self.id2w.update(
      {id: self.UNK,
       id + 1: self.SOS,
       id + 2: self.EOS}
    )

  def get_ngram_doc(self, doc: str) -> list:
    """
    Devuelve una lista con n-gramas de un documento dado.

    Args:
        doc (str): Documento del que se quieren obtener los n-gramas.

    Returns:
        list: Lista de n-gramas.
    """
    doc_tokens = self.tokenizer(doc)
    doc_tokens = self.replace_unk(doc_tokens)
    doc_tokens = [w.lower()
                  for w in doc_tokens]
    doc_tokens = [self.SOS] * (self.N - 1) + doc_tokens + [self.EOS]

    return list(nltk.ngrams(doc_tokens, self.N))

  def replace_unk(self, doc_tokens: list[str]) -> list:
    """
    Toma un lista de tokens e intercambia los tokens out-of-vocabulary por el token especial self.UNK.

    Args:
        doc_tokens (list[str]): Lista de tokens.

    Returns:
        list: Lista de tokens procesada.
    """
    for i, token in enumerate(doc_tokens):
      if token.lower() not in self.vocab:
        doc_tokens[i] = self.UNK
    return doc_tokens

  def transform(self, corpus: list[str]) -> tuple[np.ndarray, np.ndarray]:
    """
    Devuelve una tupla de arreglos de Numpy. El primero tendrá los ids de las palabras en el contexto, mientras que la segunda el id de la palabra que se debe predecir.

    Se piensa en un modelo de CBOW. Damos el contexto y queremos predecir la palabra que sigue.

    Args:
        corpus (list[str]): Lista de documentos.

    Returns:
        tuple[np.ndarray, np.ndarray]: Arreglos de numpy con ids de los contextos y id de la palabra objetivo.
    """
    X_ngrams = list()
    y = []

    for doc in corpus:
      doc_ngram = self.get_ngram_doc(doc)
      for words_window in doc_ngram:
        words_window_ids = [self.w2id[w]
                            for w in words_window]
        X_ngrams.append(list(words_window_ids[:-1]))
        y.append(words_window_ids[-1])

    return np.array(X_ngrams), np.array(y)

Transformamos nuestros datos usando la clase `NgramData`


In [6]:
tk = TweetTokenizer()

ngram_data = NgramData(args.N, 5000, tk.tokenize)
ngram_data.fit(X_train)
X_ngram_train, y_ngram_train = ngram_data.transform(X_train)
X_ngram_val, y_ngram_val = ngram_data.transform(X_val)

In [7]:
print(f"Tamaño del vocabulario : {ngram_data.get_vocab_size()}")

Tamaño del vocabulario : 5000


Se crean los objetos `TensorDataset` para guardar los datos de entrenamiento y validación.


In [8]:
# Batch size
args.batch_size = 64

# Num of workers
args.num_workers = 2

# Train
train_dataset = TensorDataset(torch.tensor(X_ngram_train, dtype=torch.int64),
                              torch.tensor(y_ngram_train, dtype=torch.int64))

train_loader = DataLoader(train_dataset,
                          batch_size=args.batch_size,
                          num_workers=args.num_workers,
                          shuffle=True)

# Val
val_dataset = TensorDataset(torch.tensor(X_ngram_val, dtype=torch.int64),
                            torch.tensor(y_ngram_val, dtype=torch.int64))

val_loader = DataLoader(val_dataset,
                        batch_size=args.batch_size,
                        num_workers=args.num_workers,
                        shuffle=True)

In [9]:
batch = next(iter(train_loader))
print(f'X shape : {batch[0].shape}')
print(f'y shape : {batch[1].shape}')

X shape : torch.Size([64, 3])
y shape : torch.Size([64])


In [10]:
batch[0]

tensor([[4998, 4997,  236],
        [  32,   26, 4997],
        [   6,  219,  162],
        [ 109,    6, 1151],
        [4998, 4998, 4998],
        [  48,  343,   45],
        [  43,    8, 1316],
        [4997, 4997,    6],
        [4997, 1792,  226],
        [4998, 4997,   48],
        [   6,    8,  570],
        [4998,  706, 4997],
        [4997, 4997, 4997],
        [3538, 4997,  166],
        [4998, 4998, 4998],
        [  14, 4997, 4997],
        [ 114,    1, 3259],
        [ 459,   51,  460],
        [  45, 4997,   83],
        [  48,  378,   48],
        [ 952,   43, 2497],
        [  60,   45, 2299],
        [ 254,   48,  256],
        [  55,   48,   20],
        [ 423, 2459,   33],
        [4998, 4998, 4998],
        [4997, 4997,   66],
        [ 941, 4997, 4997],
        [  48,  167,  129],
        [1587,  338,   16],
        [ 114,   48,   46],
        [ 990, 4997,  702],
        [4998, 4998, 4998],
        [ 112,  273, 4997],
        [1550,  193,  711],
        [ 106, 4997,

El modelo de lenguaje neuronal de Bengio c:


In [11]:
class NeuralLanguageModel(nn.Module):
  """
  Red neuronal de Bengio :)
  """

  def __init__(self, args):
    """
    Constructor  de la clase.

    El modelo de red neuronal par lenguaje de Bengio tiene la siguiente estructura:
    Para un modelo de n-gramas, se dan las primeras n-1 palabras como contexto y se intenta predecir la n-ésima palabra.
    (1) n-1 representaciones iniciales: suelen ser one-hot. Pero aquí se toman de NgramData.
        x
    (2) n-1 representaciones aprendidas de tamaño m: se obtienen de manera individual (por palabra). 
        (x = Cx)
        En esta implementación C se inicia de manera aleatoria.
    (3) Capa oculta de tamaño h: se mezclan las n-1 representaciones del paso anterior y se aplica tanh. 
        (h = tanh(Hx + d))
        Nosotros vamos a usar ReLu en vez de tanh.
    (4) Capa de salida de tamaño m: se aplica softmax a la salida de la capa anterior.
        (y = softmax(Uh + b))
        Nosotros no vamos a aplicar softmax aquí, sino afuerita.

    Args:
        args (Any): Diccionario de variables.
    """
    super(NeuralLanguageModel, self).__init__()

    self.window_size = args.N - 1  # Las n-1 palabras que entran (el contexto).
    self.embedding_size = args.m  # Tamaño de las representaciones.

    # Matriz C para convertir las representaciones. Pero está chido porque sus entradas son "entrenables".
    self.emb = nn.Embedding(args.vocab_size, args.m)
    # Primera capa oculta de las representaciones aprendidas a la oculta.
    self.fc1 = nn.Linear(args.m * (args.N - 1), args.d_h)
    # Un dropout para alocarnos
    self.drop1 = nn.Dropout(p=args.dropout)
    # Aquí solamente se va a hacer el producto por la matriz U.
    # La softmax se va a aplicar por fuera de la red para obtener la siguiente palabra según la red.
    self.fc2 = nn.Linear(args.d_h, args.vocab_size, bias=False)

  def forward(self, x):
    # Aquí se cambia la representación inicial por la aprendida.
    # Es un producto matricial. Aquí las representaciones siguen siendo matrices.
    x = self.emb(x)
    # Se cambia el tamaño para que se considere como una sola capa.
    x = x.view(-1, self.window_size * self.embedding_size)
    # Aquí se hace relu(Hx + d)
    h = F.relu(self.fc1(x))  # relu(z) = max{0, z}
    # El dropout para alocarnoooos wuuuuuuuu
    h = self.drop1(h)

    # Devolvemos solamente (Uh + b)
    return self.fc2(h)

In [12]:
def get_preds(raw_logits: torch.Tensor) -> torch.Tensor:
  """
  Aquí se toma la salida de la red neuronal (las neuronas de la última capa oculta).
  Uh + b
  Se les aplica la softmax
  softmax(Uh + b)
  Y luego se devuelve el índice de la neurona de mayor valor.

  Args:
      raw_logits (torch.Tensor | float): La salida de la red (Uh + b)

  Returns:
      torch.Tensor | int: Índice de la neurona con mayor valor después de softmax.
  """
  # Se aplica softmax.
  probs = F.softmax(raw_logits.detach(), dim=1)
  # Se obtiene el índice del valor máximo.
  y_pred = torch.argmax(probs, dim=1).cpu().numpy()

  return y_pred

In [13]:
def model_eval(data: torch.Tensor,
               model,
               gpu: bool = False) -> float | int:
  """
  Evalúa el desempeño del modelo sobre un conjunto de validación.

  Args:
      data (torch.Tensor): Conjunto de validación sobre el que se va a evaluar el modelo.
      model (_type_): Modelo que se va a evaluar.
      gpu (bool, optional): ¿Usamos gpu? Sí/No. Defaults to False.

  Returns:
      float | int: Puntaje de accuracy sobre todo el conjunto de validación.
  """
  with torch.no_grad():
    preds, tgts = list(), list()

    for window_words, labels in data:
      # windoe_words es una matriz de (64, 3)
      # labels es un vector de (64)
      if gpu:
        window_words = window_words.cuda()

      outputs = model(window_words)

      # Obrenemos las predicciones
      y_pred = get_preds(outputs)  # (64)

      tgt = labels.numpy()  # Lo que esperábamos ver (64)
      tgts.append(tgt)  # Se añade a tgts (Lista de respuestas)
      preds.append(y_pred)  # Lo que vimos (Lista de predicciones)

  # Aquí abajo "desempaquetamos" los targets y las predicciones
  # e : element
  # l : list
  # lista de todos los target
  tgts = [e for l in tgts for e in l]
  # lista de todas las predicciones
  preds = [e for l in preds for e in l]

  return accuracy_score(tgts, preds)

In [14]:
def save_checkpoint(state: dict,
                    is_best: bool,
                    checkpoint_path: str,
                    filename: str = "checkpoint.pt"):
  """
  Guarda el modelo si se ve una mejora cra. a iteraciones anteriores.

  Args:
      state (dict): Información que queremos guardar sobre el modelo.
        DEBE incluir model.state_dict()
      is_best (bool): ¿Esta versión del modelo es mejor?
      checkpoint_path (str): Directorio en el que se va a guardar el modelo.
      filename (str, optional): _description_. Defaults to "checkpoint.pt".
  """
  # Armamos la ruta para el archivo que va a guardar el modelo.
  filename = os.path.join(checkpoint_path, filename)
  # Aquí se guarda.
  torch.save(state, filename)

  # Si además es el very besto modelo ever, le ponemos un nombre que le cheque :))
  if is_best:
    shutil.copyfile(filename, os.path.join(checkpoint_path, "model_best.pt"))

Hiperparámetros de la red c:


In [15]:
# Modelo
args.vocab_size = ngram_data.get_vocab_size()
args.m = 100  # Dimensión de los embeddings de palabras
args.d_h = 200  # Dimensión de la capa oculta
args.dropout = 0.1

# Entrenamiento
args.lr = 2.3e-1
args.num_epochs = 100
args.patience = 20

# Scheduler
args.lr_patience = 10
args.lr_factor = 0.5

# Guardado de los modelos
args.savedir = 'model'
os.makedirs(args.savedir, exist_ok=True)

El scheduler sirve para


In [16]:
# Creamos el modelo
model = NeuralLanguageModel(args)

args.use_gpu = torch.cuda.is_available()
if args.use_gpu:
  model.cuda()

# Perdida, optimización y scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
  optimizer,
  "min",
  patience=args.lr_patience,
  verbose=True,
  factor=args.lr_factor
)

/home/juancho/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [17]:
start_time = time.time()

best_metric = 0
metric_history = []
train_metric_history = []

for epoch in tqdm(range(args.num_epochs),
                  desc="Epochs"):
  epoch_start_time = time.time()
  loss_epoch = []
  training_metric = []
  model.train()

  for window_words, labels in train_loader:

    # Si hay GPU
    if args.use_gpu:
      window_words = window_words.cuda()
      labels = labels.cuda()

    # Forward
    outputs = model(window_words)
    loss = criterion(outputs,
                     labels)
    loss_epoch.append(loss.item())

    # Obtener métricas de entrenamiento
    y_pred = get_preds(outputs)
    tgt = labels.cpu().numpy()
    training_metric.append(accuracy_score(tgt,
                                          y_pred))

    # Backprop y optimizamos
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Guardamos la métrica por época
  mean_epoch_metric = np.mean(training_metric)
  train_metric_history.append(mean_epoch_metric)

  # Validación para esta época
  model.eval()
  tuning_metric = model_eval(val_loader,
                             model,
                             gpu=args.use_gpu)
  metric_history.append(mean_epoch_metric)

  # Scheduler
  scheduler.step(tuning_metric)

  # Revisa si la métrica mejoró
  is_improvement = tuning_metric > best_metric
  if is_improvement:
    best_metric = tuning_metric
    n_no_improve = 0
  else:
    n_no_improve += 1

  # Si la métrica mejora, guarda el mejor modelo
  save_checkpoint(
    {
      "epoch": epoch + 1,
      "state_dict": model.state_dict(),
      "optimizer": optimizer.state_dict(),
      "scheduler": scheduler.state_dict(),
      "best_metric": best_metric
    },
    is_improvement,
    args.savedir
  )

  # Parada temprana por paciencie
  if n_no_improve >= args.patience:
    print("No improvement. Breaking out of loop.")
    break

  print(f"Train acc: {mean_epoch_metric}")
  print(f"Epoch [{epoch + 1}/{args.num_epochs}], Loss {np.mean(loss_epoch):.4f} - Val accuracy {tuning_metric:.4f} - Epoch time : {time.time() - epoch_start_time}")

print(f"--- {time.time() - start_time} seconds")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Train acc: 0.17334786636805527
Epoch [1/100], Loss 5.5255 - Val accuracy 0.2106 - Epoch time : 3.9712634086608887
Train acc: 0.18161700829550473
Epoch [2/100], Loss 5.0834 - Val accuracy 0.2051 - Epoch time : 3.347290277481079
Train acc: 0.18968497670027717
Epoch [3/100], Loss 4.8710 - Val accuracy 0.2247 - Epoch time : 3.378157377243042
Train acc: 0.19355529164825452
Epoch [4/100], Loss 4.7027 - Val accuracy 0.1987 - Epoch time : 3.3027946949005127
Train acc: 0.19839004690073514
Epoch [5/100], Loss 4.5524 - Val accuracy 0.1486 - Epoch time : 3.3078694343566895
Train acc: 0.20009139115413974
Epoch [6/100], Loss 4.4190 - Val accuracy 0.1334 - Epoch time : 3.4088571071624756
Train acc: 0.20137689310247858
Epoch [7/100], Loss 4.2941 - Val accuracy 0.2161 - Epoch time : 3.3657355308532715
Train acc: 0.2048878575302294
Epoch [8/100], Loss 4.1818 - Val accuracy 0.2211 - Epoch time : 3.3464605808258057
Train acc: 0.2091693071345358
Epoch [9/100], Loss 4.0697 - Val accuracy 0.1340 - Epoch time

In [ ]:
def print_closest_words(embeddings, ngram_data, word, n):
    '''Devuelve la lista de las n palabras mas cercanas a word'''
    word_id = torch.LongTensor([ngram_data.w2id[word]]) # obtener id de las palabras
    word_embed = embeddings(word_id) # obtener el embedding de la palabra
    dists = torch.norm(embeddings.weight - word_embed, dim=1).detach() # calcular distancias a todas las palabras
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # ordenar por distancia
    for idx, difference in lst[1:n+1]:
        print(ngram_data.id2w[idx], difference)

In [24]:
best_model = NeuralLanguageModel(args)
state_dict = torch.load("model/model_best.pt", weights_only=False)
best_model.load_state_dict(state_dict["state_dict"])
best_model.train(False)

print("-"*30)
print("Learned embeddings")
print("-"*30)
print_closest_words(best_model.emb, ngram_data, "perro", 10)

------------------------------
Learned embeddings
------------------------------
<unk> 10.232197
castiguen 10.771348
dure 10.835211
indios 10.851449
pones 10.926342
odian 10.955638
mueres 11.088757
quedado 11.160634
estuve 11.17065
holanda 11.220155


In [26]:
def parse_text(text, tokenizer):
    '''Devuelve el texto tokenizado y los ids de las palabras'''
    all_tokens = [w.lower() if w in ngram_data.w2id else '<unk>' for w in tokenizer.tokenize(text)]
    token_ids = [ngram_data.w2id[w.lower()] for w in all_tokens]
    return all_tokens, token_ids

In [27]:
def sample_next_word(logits, temperature=1.0):
    '''
    Dados los logits y la temperatura 
    (un parametro de diversidad que indica cuan determinista sera el modelo), 
    devuelve la siguiente palabra
    '''
    logits = np.asarray(logits).astype('float64')

    preds = logits/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds)
    return np.argmax(probs)

In [31]:
def predict_next_token(model, token_ids):
    word_ids_tensor = torch.LongTensor(token_ids).unsqueeze(0)
    y_raw_pred = model(word_ids_tensor).squeeze(0).detach().numpy()

    # no es necesario aplicar softmax (de aplicarse daria el mismo resultado que la prediccion cruda)
    #y_probs = F.softmax(y_raw_pred, dim=1)
    #y_pred = torch.argmax(y_probs, dim=1).detach().numpy()

    y_pred = sample_next_word(y_raw_pred, 1.0)

    return y_pred

In [54]:
def generate_text(model, initial_text, tokenizer):
    all_tokens, window_word_ids = parse_text(initial_text, tokenizer)

    for i in range(100):
        y_pred = predict_next_token(best_model, window_word_ids)
        next_word = ngram_data.id2w[y_pred]
        all_tokens.append(next_word)

        if next_word == '</s>':
            break
        else:
            window_word_ids.pop(0)
            window_word_ids.append(y_pred)

    return " ".join(all_tokens)

El modelo genera oraciones con partes coherentes, pero esto ayudara mas para calcular la probabilidad de ocurrencia de oraciones dadas.

In [67]:
initial_tokens = '<s> <s> <s>'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> <s> <unk> estoy <unk> <unk> de <unk> han <unk> hijos de portada <unk> quien chingados <unk> son drogadicto <unk> <unk> te llega pero narizon <unk> para <unk> en <unk> <unk> el ancho equivoca <unk> a <unk> <unk> <unk> cuál tipo <unk> <unk> <unk> antessss pero <unk> loca <unk> no tiene ni madres <unk> ni <unk> <unk> putos <unk> tantita madre <unk> <unk> <unk> </s>


In [72]:
initial_tokens = '<s> <s> estoy'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> <s> estoy hasta un sueño que cagado se años ejercicio el <unk> ya se <unk> más que los lea lentes les encanta <unk> <unk> qué me pones mucho <unk> porfa 😌 que soy feliz ni loca <unk> yo sigo </s>


In [69]:
initial_tokens = '<s> saludos a'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
<s> saludos a cruda ahi <unk> llegó han por eso todo el mundo para meter más dj te <unk> aún <unk> <unk> <unk> fundadora </s>


In [73]:
initial_tokens = 'yo opino que'
print('-'*30)
print("Learned embeddings")
print('-'*30)
print(generate_text(best_model, initial_tokens, tk))

------------------------------
Learned embeddings
------------------------------
yo opino que despertar <unk> d yo dinero y el <unk> cancelar de <unk> <unk> madre <unk> una vez le enseña caso <unk> <unk> <unk> se <unk> 😍 <unk> </s>


In [74]:
def log_likelihood(model, test, ngram_model):
    # Generar n gram windows from input text and the respective label y
    X, y = ngram_model.transform(test)
    # discard first two n-gram windows since they may contain <s> tokens (not necessary)
    X, y = X[2:], y[2:]
    X = torch.LongTensor(X).unsqueeze(0)

    logits = model(X).detach()
    probs = F.softmax(logits, dim=1).numpy()

    return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))

Se observa como cambian los valores para oraciones menos probables de observar

In [75]:
print("log likelihood: ", log_likelihood(best_model, "Estamos en la clase de procesamiento de lenguaje", ngram_data))

log likelihood:  -771.2288104891777


/tmp/ipykernel_10181/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


In [76]:
print("log likelihood: ", log_likelihood(best_model, "Estamos procesamiento clase en la de natural lenguaje", ngram_data))

log likelihood:  -859.9079782366753


/tmp/ipykernel_10181/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


In [77]:
print("log likelihood: ", log_likelihood(best_model, " la natural Estamos clase en de de lenguaje procesamiento", ngram_data))

log likelihood:  -916.2573439478874


/tmp/ipykernel_10181/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


# Estructuras sintacticas correctas

In [82]:
from itertools import permutations
from random import shuffle

word_list = "sino gano me voy a la chingada".split(" ")
perms = [' '.join(p) for p in permutations(word_list)]
#print(perms)
print('-'*30)

for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[:5]:
    print(p, t)

print('-'*30)
for p, t in sorted([(log_likelihood(best_model, text, ngram_data), text) for text in perms], reverse=True)[-5:]:
    print(p, t)

------------------------------


/tmp/ipykernel_10181/971388848.py:11: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.log(probs[i][w]) for i, w in enumerate(y))


-448.9200522303581 me voy sino la a gano chingada
-448.9200522303581 me voy sino a la gano chingada
-448.9200522303581 me voy sino a gano la chingada
-448.9200522303581 me voy sino a gano chingada la
-448.9200522303581 me voy la sino a gano chingada
------------------------------
-453.9536334872246 chingada gano me la a sino voy
-453.9536334872246 chingada gano me a voy la sino
-453.9536334872246 chingada gano me a sino la voy
-453.9536334872246 chingada gano me a la voy sino
-453.9536334872246 chingada gano me a la sino voy
